In [ ]:
#################################################
##########  (c) by Alfred Galichon     ##########
#################################################

## setting up the data

In [1]:
install.packages("slam")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [1]:
install.packages('/Library/gurobi752/mac64/R/gurobi_7.5-2.tgz', repos=NULL)

In [1]:
library(gurobi)
setwd('~/Dropbox/matheconcode/_applications/01-appli-lp/')
thepath = getwd()
filename="/StiglerData1939.txt"
thedata = as.matrix(read.csv(paste0(thepath,filename),sep="\t", header=T))
nbCommodities=length(which(thedata[,1] != "" ))-1
names = thedata[1:nbCommodities,1]
themat = matrix( as.numeric(thedata[,3:13]), ncol = 11)
themat[is.na(themat)] = 0

Loading required package: slam


## calling Gurobi

In [ ]:
N = t(themat[1:nbCommodities,3:11])
d = themat[(nbCommodities+1),3:11]
c = rep(1,nbCommodities)
result   = gurobi ( list(A=N,obj=c,modelsense="min",rhs=d,sense=">"), params=list(OutputFlag=0) ) 
q_yearly  = result$x * 365 # convert into yearly cost
pi = result$pi
cost_daily =  result$objval

## display optimal solution

In [ ]:
print("*** optimal solution ***")
toKeep = which(q_yearly !=0 )
foods = q_yearly[toKeep]
names(foods) = names[toKeep]
print(foods)
print(paste0("Total cost (optimal)= ", sum(q_yearly*c) ))

## compare with Stigler's solution

In [ ]:
print("*** Stigler's solution ***")
toKeepStigler = c(1,15,46,52,69)
foods_stigler = c(13.33, 3.84,4.11,1.85,16.80)
names(foods_stigler) = names[toKeepStigler]
print(foods_stigler)
print(paste0("Total cost (Stigler)= ", sum(foods_stigler*c[toKeepStigler]) ))

## alternatively, use glpk

In [ ]:
print("*** Optimal solution using Rglpk ***")
library(Rglpk)
resGlpk = Rglpk_solve_LP(obj=c, mat=N, dir=rep(">",length(d)), rhs=d, bounds = NULL, 
                         max = FALSE, control = list())
print(resGlpk$optimum*365)